In [1]:
# https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
%load_ext autoreload
# Reload all modules (except those excluded by %aimport) every time before executing the Python code typed:
%autoreload 2

In [2]:
# General imports
from pathlib import Path
import numpy as np
import xarray as xr

# Plotting
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20, 10)
plt.rcParams["figure.facecolor"] = "white"

# power_perceiver imports
from power_perceiver.data_loader import HRVSatellite, GSP, PV, Sun
from power_perceiver.dataset import NowcastingDataset
from power_perceiver.transforms.pv import PVPowerRollingWindow
from power_perceiver.transforms.satellite import PatchSatellite
from power_perceiver.xr_batch_processor import ReduceNumPVSystems, SelectPVSystemsNearCenterOfImage
from power_perceiver.np_batch_processor import EncodeSpaceTime, Topography
from power_perceiver.consts import BatchKey



/home/jack/miniconda3/envs/power_perceiver/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
DATA_PATH = Path("~/dev/ocf/power_perceiver/data_for_testing/").expanduser()
assert DATA_PATH.exists()

In [4]:
dataset = NowcastingDataset(
    data_path=DATA_PATH,
    data_loaders=[
        HRVSatellite(
            transforms=[
                PatchSatellite(),
            ]
        ),
        PV(transforms=[PVPowerRollingWindow()]),
        GSP(),
    ],
    xr_batch_processors=[
        SelectPVSystemsNearCenterOfImage(),
        ReduceNumPVSystems(requested_num_pv_systems=8),
    ],
    np_batch_processors=[
        EncodeSpaceTime(),
        Topography("/home/jack/europe_dem_2km_osgb.tif"),
    ],
)


/home/jack/miniconda3/envs/power_perceiver/lib/python3.9/site-packages/pyresample/image.py:59: FutureWarning: Usage of ImageContainer is deprecated, please use NumpyResamplerBilinear class instead
  warnings.warn(
/home/jack/miniconda3/envs/power_perceiver/lib/python3.9/site-packages/pyproj/crs/crs.py:1256: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  return self._crs.to_proj4(version=version)
/home/jack/miniconda3/envs/power_perceiver/lib/python3.9/site-packages/pyproj/crs/crs.py:1256: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  return self._crs.to_proj4(version=version)
/home/jack/miniconda3/envs/power_perceiver/lib/python3.9/site-pack

In [5]:
np_batch = dataset[0]

In [6]:

np_batch.keys()



dict_keys([<BatchKey.hrvsatellite: 1>, <BatchKey.hrvsatellite_time_utc: 6>, <BatchKey.hrvsatellite_y_osgb: 2>, <BatchKey.hrvsatellite_x_osgb: 3>, <BatchKey.hrvsatellite_y_geostationary: 4>, <BatchKey.hrvsatellite_x_geostationary: 5>, <BatchKey.pv: 11>, <BatchKey.pv_system_row_number: 12>, <BatchKey.pv_id: 13>, <BatchKey.pv_capacity_wp: 14>, <BatchKey.pv_mask: 15>, <BatchKey.pv_time_utc: 18>, <BatchKey.pv_x_osgb: 17>, <BatchKey.pv_y_osgb: 16>, <BatchKey.gsp: 22>, <BatchKey.gsp_id: 23>, <BatchKey.gsp_time_utc: 26>, <BatchKey.gsp_y_osgb: 24>, <BatchKey.gsp_x_osgb: 25>, <BatchKey.hrvsatellite_x_osgb_fourier: 9>, <BatchKey.pv_x_osgb_fourier: 20>, <BatchKey.gsp_x_osgb_fourier: 28>, <BatchKey.hrvsatellite_y_osgb_fourier: 8>, <BatchKey.pv_y_osgb_fourier: 19>, <BatchKey.gsp_y_osgb_fourier: 27>, <BatchKey.hrvsatellite_time_utc_fourier: 10>, <BatchKey.pv_time_utc_fourier: 21>, <BatchKey.gsp_time_utc_fourier: 29>, <BatchKey.hrvsatellite_surface_height: 7>])

In [7]:
np_batch[BatchKey.gsp].shape

(31, 6)

In [9]:
np_batch[BatchKey.gsp_x_osgb_fourier]

array([[[ 0.96334934,  0.2682499 ,  0.51683676, ..., -0.7275389 ,
         -0.8849113 ,  0.46575958],
        [ 0.2685518 ,  0.96326524,  0.51737326, ...,  0.6853822 ,
          0.88549453,  0.46464986],
        [ 0.53803307, -0.84292376, -0.90704167, ...,  0.13311297,
         -0.7638033 , -0.6454491 ],
        ...,
        [-0.9677916 , -0.2517527 ,  0.48728827, ...,  0.69143426,
         -0.85104036,  0.5251003 ],
        [ 0.96334934,  0.2682499 ,  0.51683676, ..., -0.7275389 ,
         -0.8849113 ,  0.46575958],
        [ 0.87547547,  0.48326254,  0.846169  , ..., -0.99833786,
         -0.9018717 , -0.43200403]],

       [[ 0.7956833 , -0.60571283, -0.9639112 , ...,  0.9282233 ,
          0.51323265, -0.85824955],
        [ 0.9298948 ,  0.3678256 ,  0.6840782 , ..., -0.9044159 ,
         -0.99794513,  0.064074  ],
        [-0.34678057, -0.93794626,  0.6505231 , ..., -0.4867696 ,
          0.988127  ,  0.1536394 ],
        ...,
        [-0.785219  ,  0.6192181 , -0.9724437 , ..., -

In [13]:
gsp = GSP(data_path=DATA_PATH)[0]
gsp


<xarray.Dataset>
Dimensions:           (example: 32, time: 6, gsp: 32)
Coordinates:
    gsp_id            (example, gsp) float32 332.0 159.0 171.0 ... nan nan nan
    time_utc          (example, time) datetime64[ns] 2020-09-12T09:00:00 ... ...
    x_osgb            (example, gsp) float32 2.22e+05 2.787e+05 ... nan nan
    y_osgb            (example, gsp) float32 7.157e+05 6.921e+05 ... nan nan
Dimensions without coordinates: example, time, gsp
Data variables:
    capacity_mwp      (example, time, gsp) float32 0.3418 7.573 ... 0.0 0.0
    power_mw          (example, time, gsp) float32 0.006855 3.144 ... 0.0 0.0
    power_normalised  (example, time, gsp) float32 0.02005 0.4152 ... nan nan

In [14]:
gsp.isel(gsp=0)

<xarray.Dataset>
Dimensions:           (example: 32, time: 6)
Coordinates:
    gsp_id            (example) float32 332.0 120.0 64.0 ... 43.0 332.0 129.0
    time_utc          (example, time) datetime64[ns] 2020-09-12T09:00:00 ... ...
    x_osgb            (example) float32 2.22e+05 1.828e+05 ... 4.532e+05
    y_osgb            (example) float32 7.157e+05 5.16e+04 ... 7.157e+05 3.9e+05
Dimensions without coordinates: example, time
Data variables:
    capacity_mwp      (example, time) float32 0.3418 0.3418 ... 17.13 17.13
    power_mw          (example, time) float32 0.006855 0.06283 ... 7.415 5.45
    power_normalised  (example, time) float32 0.02005 0.1838 ... 0.433 0.3182

In [4]:
hrv_satellite = HRVSatellite(data_path=DATA_PATH)

In [19]:
hrv = hrv_satellite[0]
hrv

<xarray.Dataset>
Dimensions:          (example: 32, channel: 1, time: 31, y: 64, x: 64)
Coordinates:
    channel_name     (example, channel) object 'HRV' 'HRV' 'HRV' ... 'HRV' 'HRV'
    time_utc         (example, time) datetime64[ns] 2020-09-12T09:05:00 ... 2...
    x_geostationary  (example, x) float32 -8.461e+05 -8.451e+05 ... -6.281e+05
    x_osgb           (example, y, x) float32 2.007e+05 2.018e+05 ... 4.753e+05
    y_geostationary  (example, y) float32 4.818e+06 4.819e+06 ... 4.75e+06
    y_osgb           (example, y, x) float32 6.382e+05 6.381e+05 ... 4.567e+05
Dimensions without coordinates: example, channel, time, y, x
Data variables:
    data             (example, time, channel, y, x) int16 260 260 ... 139 138

In [16]:
gsp = GSP(data_path=DATA_PATH)[0]
gsp

<xarray.Dataset>
Dimensions:           (example: 32, time: 6, gsp: 32)
Coordinates:
    gsp_id            (example, gsp) float32 332.0 159.0 171.0 ... nan nan nan
    time_utc          (example, time) datetime64[ns] 2020-09-12T09:00:00 ... ...
    x_osgb            (example, gsp) float32 2.22e+05 2.787e+05 ... 0.0 0.0
    y_osgb            (example, gsp) float32 7.157e+05 6.921e+05 ... 0.0 0.0
Dimensions without coordinates: example, time, gsp
Data variables:
    capacity_mwp      (example, time, gsp) float32 0.3418 7.573 ... 0.0 0.0
    power_mw          (example, time, gsp) float32 0.006855 3.144 ... 0.0 0.0
    power_normalised  (example, time, gsp) float32 0.02005 0.4152 ... nan nan

In [23]:
gsp["time_utc"].isel(time=0) <= hrv["time_utc"].isel(time=0)

<xarray.DataArray 'time_utc' (example: 32)>
array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])
Dimensions without coordinates: example

In [27]:
gsp["time_utc"].isel(time=-1) <= hrv["time_utc"].isel(time=-1)

<xarray.DataArray 'time_utc' (example: 32)>
array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])
Dimensions without coordinates: example

In [28]:
gsp["time_utc"].isel(time=-1)

<xarray.DataArray 'time_utc' (example: 32)>
array(['2020-09-12T11:30:00.000000000', '2020-09-22T13:00:00.000000000',
       '2020-06-09T09:00:00.000000000', '2020-10-01T12:00:00.000000000',
       '2020-05-02T09:00:00.000000000', '2020-05-24T10:00:00.000000000',
       '2020-09-10T15:30:00.000000000', '2020-05-13T13:30:00.000000000',
       '2020-08-25T14:30:00.000000000', '2020-05-19T14:00:00.000000000',
       '2020-10-15T14:30:00.000000000', '2020-08-24T12:30:00.000000000',
       '2020-11-19T12:30:00.000000000', '2020-10-21T13:30:00.000000000',
       '2020-06-27T14:00:00.000000000', '2020-07-16T12:00:00.000000000',
       '2020-08-21T14:30:00.000000000', '2020-04-20T15:00:00.000000000',
       '2020-04-20T09:30:00.000000000', '2020-07-05T10:30:00.000000000',
       '2020-08-30T12:00:00.000000000', '2020-07-05T13:30:00.000000000',
       '2020-10-04T16:00:00.000000000', '2020-10-20T14:00:00.000000000',
       '2020-07-17T15:00:00.000000000', '2020-06-27T12:00:00.000000000',
       '2020-10-16T15:00:00.000000000', '2020-04-09T15:30:00.000000000',
       '2020-12-05T13:30:00.000000000', '2020-08-14T11:30:00.000000000',
       '2020-08-30T15:00:00.000000000', '2020-06-17T15:30:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
    time_utc  (example) datetime64[ns] 2020-09-12T11:30:00 ... 2020-06-17T15:...
Dimensions without coordinates: example

In [29]:
hrv["time_utc"].isel(time=-1)

<xarray.DataArray 'time_utc' (example: 32)>
array(['2020-09-12T11:35:00.000000000', '2020-09-22T13:10:00.000000000',
       '2020-06-09T09:05:00.000000000', '2020-10-01T12:00:00.000000000',
       '2020-05-02T09:00:00.000000000', '2020-05-24T10:05:00.000000000',
       '2020-09-10T15:50:00.000000000', '2020-05-13T13:55:00.000000000',
       '2020-08-25T14:40:00.000000000', '2020-05-19T14:05:00.000000000',
       '2020-10-15T14:50:00.000000000', '2020-08-24T12:50:00.000000000',
       '2020-11-19T12:40:00.000000000', '2020-10-21T13:40:00.000000000',
       '2020-06-27T14:20:00.000000000', '2020-07-16T12:00:00.000000000',
       '2020-08-21T14:55:00.000000000', '2020-04-20T15:25:00.000000000',
       '2020-04-20T09:35:00.000000000', '2020-07-05T10:55:00.000000000',
       '2020-08-30T12:00:00.000000000', '2020-07-05T13:35:00.000000000',
       '2020-10-04T16:20:00.000000000', '2020-10-20T14:25:00.000000000',
       '2020-07-17T15:10:00.000000000', '2020-06-27T12:20:00.000000000',
       '2020-10-16T15:10:00.000000000', '2020-04-09T15:40:00.000000000',
       '2020-12-05T13:30:00.000000000', '2020-08-14T11:40:00.000000000',
       '2020-08-30T15:20:00.000000000', '2020-06-17T15:45:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
    time_utc  (example) datetime64[ns] 2020-09-12T11:35:00 ... 2020-06-17T15:...
Dimensions without coordinates: example

In [81]:
start_idx_5_min = 2
start_idx_30_min = int(np.ceil(start_idx_5_min / 6)) + 1

EXAMPLE_IDX = 1

(
    start_idx_30_min, 
    hrv["time_utc"].isel(example=EXAMPLE_IDX, time=start_idx_5_min), 
    gsp["time_utc"].isel(example=EXAMPLE_IDX, time=start_idx_30_min),
    hrv["time_utc"].isel(example=EXAMPLE_IDX, time=start_idx_5_min + 9), 
)

(2,
 <xarray.DataArray 'time_utc' ()>
 array('2020-09-22T10:50:00.000000000', dtype='datetime64[ns]')
 Coordinates:
     time_utc  datetime64[ns] 2020-09-22T10:50:00,
 <xarray.DataArray 'time_utc' ()>
 array('2020-09-22T11:30:00.000000000', dtype='datetime64[ns]')
 Coordinates:
     time_utc  datetime64[ns] 2020-09-22T11:30:00,
 <xarray.DataArray 'time_utc' ()>
 array('2020-09-22T11:35:00.000000000', dtype='datetime64[ns]')
 Coordinates:
     time_utc  datetime64[ns] 2020-09-22T11:35:00)

In [ ]:
# TODO: Do GSPs with NaN gsp_id also have 0 capacity?